In [ ]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import operator
from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('english'))
from textblob import TextBlob

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
# 1. Random Forest
from sklearn.ensemble import (RandomForestClassifier,
                              GradientBoostingClassifier,
                              AdaBoostClassifier)

from sklearn.metrics import accuracy_score
import time

def rf(X_dev,y_dev,X_eval,y_eval,dict_clf):
    paramgrid = {
        'n_estimators':      [100, 200, 500, 750],
        'criterion':         ['gini', 'entropy'],
        'max_features':      ['auto', 'log2'],
        'min_samples_leaf':  list(range(2, 5))
    }
    GS = GridSearchCV(RandomForestClassifier(random_state=77),
                      paramgrid,
                      cv=4)
    t0 = time.time()
    GS.fit(X_dev, y_dev)
    t = time.time() - t0
    best_clf = GS.best_estimator_
    best_params = GS.best_params_
    best_score = GS.best_score_
    name = 'RF'
    best_clf.fit(X_dev, y_dev)
    acc_eval = accuracy_score(y_eval, best_clf.predict(X_eval))
    dict_clf[name] = {
        'best_par': best_params,
        'best_clf': best_clf,
        'best_score': best_score,
        'score_eval': acc_eval,
        'fit_time': t,
    }
    
    return dict_clf

# 2. GradientBoosting
def gb(X_dev,y_dev,X_eval,y_eval,dict_clf):
    paramgrid = {
        'n_estimators':      [100, 200, 500, 750],
        'max_features':      ['auto', 'log2'],
        'min_samples_leaf':  list(range(2, 7)),
        'loss' :             ['deviance'],
        'learning_rate':     [0.05, 0.1, 0.2],
    }
    GS = GridSearchCV(GradientBoostingClassifier(random_state=77),
                      paramgrid,
                      cv=4)
    t0 = time.time()
    GS.fit(X_dev, y_dev)
    t = time.time() - t0
    best_clf = GS.best_estimator_
    best_params = GS.best_params_
    best_score = GS.best_score_
    name = 'GB'
    best_clf.fit(X_dev, y_dev)
    acc_eval = accuracy_score(y_eval, best_clf.predict(X_eval))
    dict_clf[name] = {
        'best_par': best_params,
        'best_clf': best_clf,
        'best_score': best_score,
        'score_eval': acc_eval,
        'fit_time': t,
    }
    
    return dict_clf
# 3. AdaBoost
def ab(X_dev,y_dev,X_eval,y_eval,dict_clf):

    paramgrid = {
        'n_estimators':  [100, 200, 500, 750],
        'learning_rate': [0.05, 0.1, 0.5, 1, 2]
    }
    GS = GridSearchCV(AdaBoostClassifier(random_state=77),
                      paramgrid,
                      cv=4)
    t0 = time.time()
    GS.fit(X_dev, y_dev)
    t = time.time() - t0
    best_clf = GS.best_estimator_
    best_params = GS.best_params_
    best_score = GS.best_score_
    name = 'ADB'
    best_clf.fit(X_dev, y_dev)
    acc_eval = accuracy_score(y_eval, best_clf.predict(X_eval))
    dict_clf[name] = {
        'best_par': best_params,
        'best_clf': best_clf,
        'best_score': best_score,
        'score_eval': acc_eval,
        'fit_time': t,
    }
    
    return dict_clf

def train_it(X_train,y_train_internal, X_test,  y_test_internal,main_test):
    dict_clf = {}
    dict_clf = rf(X_train,y_train_internal, X_test,  y_test_internal,dict_clf)
    dict_clf = gb(X_train,y_train_internal, X_test,  y_test_internal,dict_clf)
    dict_clf =ab(X_train,y_train_internal, X_test,  y_test_internal,dict_clf)
    from sklearn.ensemble import VotingClassifier

    estimators = [('RF', dict_clf['RF']['best_clf']),
                  ('GB', dict_clf['GB']['best_clf']),
                  ('ADB', dict_clf['ADB']['best_clf'])]

    # Instanciate the VotingClassifier using the soft voting
    voter = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)
    voter.fit(X_train, y_train_internal)

    pred = voter.predict(main_test).astype(int)
    return pred

In [ ]:
def get_ner_tags(sentences):
    all_sample_tags = []
    for i in sentences:
        dict_count = {}
        doc = nlp(i)
        result = ([(X.label_) for X in doc.ents])

        for tag in result:
            if tag not in dict_count:
                dict_count[tag] = 1
            else:
                dict_count[tag]  += 1
        all_sample_tags.append(dict_count)
    
    cc = pd.DataFrame.from_dict(all_sample_tags)
    cc.fillna(0,inplace=True)
    return cc
        
    
    

In [ ]:
def get_part_of_speech_tags(sentences):
    all_sample_tags = []
    for i in sentences:
        dict_count = {}
        result = (TextBlob(i))
        for words, tag in result.tags:
            if tag not in dict_count:
                dict_count[tag] = 1
            else:
                dict_count[tag]  += 1
        all_sample_tags.append(dict_count)
    
    cc = pd.DataFrame.from_dict(all_sample_tags)
    cc.fillna(0,inplace=True)
    return cc


def get_sen_len(sentences):
    sen_len = []
    for i in sentences:
        i = len(i.split(' '))
        sen_len.append(i)
    return sen_len


def get_noun_phrase(sentences):
    l = []
    for i in sentences:
        doc = nlp(i)
        cnt = 0
        for chunk in doc.noun_chunks:
            cnt +=1
        l.append(cnt)
    return l 

def pca(X):
    import numpy as np
    from sklearn.decomposition import PCA
    pca = PCA(n_components=100)
    pc = pca.fit_transform(X)
    return pc

def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(str.maketrans('', '', string.punctuation))
    filter_str = final_str.translate(str.maketrans('', '', digits))
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()


def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:   
                get_word_vec = model[w]
            except:
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen


#vocab = model.keys()

df_test = pd.read_csv('test_dataset.csv')
df_test.head()
df_train = pd.read_csv('train_dataset.csv')

df_train['source_'] = 1
df_test['source_'] =0

df = pd.concat([df_train,df_test])
df['score_1'].fillna(1, inplace = True)
df['score_2'].fillna(1, inplace = True)
df['score_3'].fillna(1, inplace = True)
df['score_4'].fillna(1, inplace = True)
df['score_5'].fillna(1, inplace = True)

df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)

multi_datasets = {}
groups = df.groupby('Essayset')
for name, group in groups:
    multi_datasets[name] = group

pp = []
def different_models(model,multi_datasets):
    flag = 1
    for i in multi_datasets:
        print(i)
        
        single_df = multi_datasets[i]
        single_df.reset_index(inplace= True)
        single_df.dropna(axis=0,inplace=True)

        test_id = list(single_df.loc[single_df['source_'] ==0]['ID'])
        es = list(single_df.loc[single_df['source_'] ==0]['Essayset'])
        if len(test_id) != len(es):
            print('PANGA')
            break

        single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
        single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
        single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)

        single_df['kitna_aacha']=single_df['candi_score']

        single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)

        #all_text  = []
        
        
        parts_dataset = get_part_of_speech_tags(list(single_df['EssayText']))
        ner_tags = get_ner_tags(list(single_df['EssayText']))
        #noun_p = get_noun_phrase(list(single_df['EssayText']))
        single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
        
        sen_len = get_sen_len(list(single_df['EssayText']))
        
        #all_text = list(single_df['EssayText'])
        vectors = get_vector(model,list(single_df['EssayText']))

        #from sklearn.feature_extraction.text import TfidfVectorizer
        #tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
        #tfidf.fit(all_text)

        #features = tfidf.transform(single_df['EssayText'])

        new_tfidf_features = pd.DataFrame(vectors)
        new_tfidf_features['clarity'] = list(single_df['clarity'])
        new_tfidf_features['sen_len'] = sen_len
        #new_tfidf_features['noun_p'] = noun_p
        new_tfidf_features['coherent'] = list(single_df['coherent'])
        new_tfidf_features['kitna_aacha'] = list(single_df['kitna_aacha'])
        new_tfidf_features['source_'] = list(single_df['source_'])
        
        
        new_tfidf_features = pd.concat([new_tfidf_features,parts_dataset],axis=1)
        
        new_tfidf_features.dropna(axis=0,inplace=True)
        all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity','coherent'])
        preprocessed_main_test = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 0]
        preprocessed_main_train = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 1]

        preprocessed_main_test.drop(['source_','kitna_aacha'],axis =1,inplace=True)

        y_train = (preprocessed_main_train['kitna_aacha'])
        x_train = preprocessed_main_train.drop(['kitna_aacha','source_'],axis =1)
        
        from sklearn.model_selection import train_test_split
        from sklearn.naive_bayes import MultinomialNB,GaussianNB
        from sklearn.linear_model import LogisticRegression
        X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.10)
    
        num_cla = len(set(y_train))
        print('----')
        print(num_cla)
        pred_internal = train_it(X_train,y_train_internal, X_test,  y_test_internal,preprocessed_main_test)

        res =list(zip(test_id,es,pred_internal))
        pp.append(res)
    return pp
    #res =list(zip(test_id,pred_internal))



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [ ]:
X_train, X_test, y_train_internal, y_test_internal = different_models(model,multi_datasets)

In [ ]:
a = pp

In [ ]:
from itertools import chain
import itertools
l = list(itertools.chain(*a))

In [ ]:
res_ = pd.DataFrame(l,columns=['id','essay_set','essay_score'])

In [ ]:
res_.to_csv('word_embed_vote_3_mode_long.csv',index=False)

In [ ]:
# df_test.heada
X_train

In [ ]:
# df_test['EssayText'][22]

In [ ]:
# import operator
# l  = [[9.99690792e-01 ,1, 1.51614726e-07, 4.23026149e-13],[1,2,3,4]]
# index, value = max(enumerate(l), key=operator.itemgetter(1))

In [ ]:
# pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],l))

In [ ]:
# doc = nlp("")
# cnt = 0
# for chunk in doc.noun_chunks:
#     cnt +=1
# print(cnt)

In [ ]:
#78.7

# 'num_leaves': 15,
# 'max_bin': 100,
# 'num_class':num_cla,
# 'min_data_in_leaf': 50,
# 'learning_rate': 0.01,
# 'min_sum_hessian_in_leaf': 0.000446,
# 'bagging_fraction': 0.55,
# 'bagging_freq': 5,
# 'max_depth': 14,
# 'save_binary': True,
# 'seed': 31452,
# 'feature_fraction_seed': 31415,
# 'feature_fraction': 0.51,
# 'bagging_seed': 31415,
# 'drop_seed': 31415,
# 'data_random_seed': 31415,
# 'objective': 'multiclass',
# 'boosting_type': 'gbdt',
# 'verbose': 1,htpo
# 'metric': 'multi_logloss',

# 'is_unbalance': False,

In [ ]:

def d2(dic):
    dic[1] = 0
    return dic
def d1():
    dic = {}
    dic = d2(dic)
    dic = d3(dic)
    return(dic)

def d3(dic):
    dic[2] =12
    return dic

In [ ]:
d1()